# Изучаем Python и OpenCV

## Python basics

In [ ]:
#это комментарий

a = "это переменная"

In [ ]:
print(a)

In [ ]:
print(b)

In [ ]:
if a == 0:
    print("!")

In [ ]:
b = ["это", "список", "строк"]

#чтобы что-то распечаать в любом месте программы можно написать
print(b) # здесь print это функция от аргумента b

In [ ]:
#определим свою функцию:
def greeting(name):
    result = "Привет! Я {}".format(name) 
    return result # метод формат позволяет подставить аргумент в строку

greeting("художник")

In [ ]:
# функция может брать на вход несколько аргументов и возвращать несколько значений
def sum_and_mult(a, b, c=10):
    return a + b + c, a * b * c

s, m = sum_and_mult(1, 2)
print(s, m)

## Библиотека NumPy

In [ ]:
# подключаем
import numpy as np

In [ ]:
# это список:
a = [1, 2, 3]

# а это - массив
new_a = np.array(a)
new_a

In [ ]:
# это список из списков
mat = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# а это уже двумерный массив
new_mat = np.array(mat)
new_mat

Обращаться к элементам массива можно как и к элементам списков по индексам:

In [ ]:
print(mat[1][1])

Но!

In [ ]:
print(new_mat[1, 1])

## Библиотека OpenCV

### Работаем с камерой

Чтобы понять, доступна ли вам камера ноутбука или внешняя USB камера по стандартному интерфейсу, можно найти на компьютере файл /dev/video0

https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html

In [ ]:
!ls /dev/video*

In [ ]:
import matplotlib.pyplot as plt # эта библиотека для визулизации
# хотим картинки прямо тут
%matplotlib inline
import cv2
import time

cap = cv2.VideoCapture(0)

if cap.isOpened():
    time.sleep(1)
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame)
else:
    print("ЧТО-ТО ПОШЛО НЕ ТАК!!!")

cap.release()

In [ ]:
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

## Face Detection using dlib

using https://hackernoon.com/gender-and-race-change-on-your-selfie-with-neural-nets-9a9a1c9c5c16

In [ ]:
!pip install dlib

Для удобства сделаем функцию make_shot

In [ ]:
def make_shot():
    cap = cv2.VideoCapture(0)
    if cap.isOpened():
        time.sleep(1)
        ret, _frame = cap.read()
        _frame = cv2.cvtColor(_frame, cv2.COLOR_BGR2RGB)
        plt.imshow(_frame)
    else:
        print("ЧТО-ТО ПОШЛО НЕ ТАК!!!")
    cap.release()
    return _frame

frame = make_shot()

In [ ]:
import dlib

fd_front_dlib = dlib.get_frontal_face_detector()
faces = fd_front_dlib(frame, 1)
print(faces)

Детектор мог ничего и не вернуть, попробуем запустить его в сыром режиме и посмотреть на все, даже самые плохие детекты

In [ ]:
dets, scores, idx = fd_front_dlib.run(frame, 1, -1)

In [ ]:
print(dets, scores)

Функция detect_single_face_dlib возвращает координаты прямоугольника описывающего лицо, попробуем его нарисовать поверх картинки. Для рисования поверх картинок в OpenCV есть много функций, описанных здесь https://docs.opencv.org/3.0-beta/modules/imgproc/doc/drawing_functions.html

нарисуем какой-нибудь один

In [ ]:
d = dets[0]

cv2.rectangle(frame,
              (d.left(), d.top()),
              (d.right(), d.bottom()),
              (255, 0, 0), thickness=5)
plt.imshow(frame)

Для удобства для рисования детекций заведем функцию:

In [ ]:
def draw(input_img, detect):
    img = cv2.rectangle(input_img,
                        (detect.left(), detect.top()),
                        (detect.right(), detect.bottom()),
                        (255, 0, 0), thickness=5)
    return img

Нарисуем все детекты

In [ ]:
new_img = frame.copy()
for det in dets:
    new_img = draw(new_img, det)
plt.imshow(new_img)

добавим возможность выводить уверенность детекта (используем https://docs.opencv.org/3.0-beta/modules/imgproc/doc/drawing_functions.html#puttext):

In [ ]:
def draw(input_img, detect, score):
    img = cv2.rectangle(input_img,
                        (detect.left(), detect.top()),
                        (detect.right(), detect.bottom()),
                        (255, 0, 0), thickness=5)
    cv2.putText(input_img, "%.4f" % score, (detect.left(), detect.top()),
                cv2.FONT_HERSHEY_SIMPLEX, 2, 255, 5)
    return img

In [ ]:
new_img = frame
for i in range(len(dets)):
    new_img = draw(new_img, dets[i], scores[i])
plt.imshow(new_img)

В потоке:

In [ ]:
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #BGR -> RGB

        dets, scores, idx = fd_front_dlib.run(frame, 1, -1) # detect
        
        for i in range(len(dets)): # visualize
            new_img = draw(frame, dets[i], scores[i])

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) #RGB -> BGR
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

# Что дальше?

используя https://github.com/ageitgey/face_recognition и захват с камеры, сделать демо с распознаванием лиц одногруппников